# nlp_q.ipynb  FoodBigrams

  Return the top 100 bigrams based on the probability statistic
  
  $${ p(w_1 w_2) \over{p(w_1) \cdot p(w_2)} } = {p(bigram)\over{p(word1) \cdot p(word2)}}$$




In [1]:
import simplejson as json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
pd.options.display.max_rows = 999

print 'success'

success


In [17]:
## Get all restaurant IDs ###

lst_id_restaurant = []
# get restaurant IDs from the ml json file
with open("/home/vagrant/miniprojects/questions/2_week/ml/data/yelp_train_academic_dataset_business.json") as f:
    for line in f:
        tmp = json.loads(line)
        if 'Restaurants' or 'Food' in tmp['categories']:
            lst_id_restaurant.append(tmp['business_id'])
# Of 37938 Reviews, 12876 are Restaurants
print len(lst_id_restaurant), 'restaurants'

37938 restaurants


In [18]:
## Subset reviews for restaurants

lst_reviews = []
lst_stars = []
# get reviews for restaurants
with open("/home/vagrant/miniprojects/questions/3_week/nlp/data/yelp_train_academic_dataset_review.json") as f:
    for line in f:
            tmp = json.loads(line)
            if tmp['business_id'] in lst_id_restaurant:
                lst_reviews.append(tmp['text'])
                #lst_stars.append(tmp['stars'])

              
print len(lst_id_restaurant), 'restaurants'
print len(lst_reviews), "restaurant reviews"

del lst_id_restaurant  

37938 restaurants
910484 restaurant reviews


In [21]:
## Store Huge Honking Review List ###
joblib.dump(lst_reviews, "/home/vagrant/miniprojects/questions/3_week/nlp/pickles/nlp_lst_restfood.pkl")

['/home/vagrant/miniprojects/questions/3_week/nlp/pickles/nlp_lst_restfood.pkl']

In [5]:
## Define stopwords ###

# import nltk
# stops = nltk.corpus.stopwords.words('english')

# nltk corpus may not be available on heroku:
stops = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now']

In [22]:
## Train Unigram + Bigram CountVectorizers ###
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer

parameters = {'stop_words':stops, 'min_df':2, 'max_df':1.0}  # min_df=2 filters out garbage

# make unigram counter
ug_vec = CountVectorizer(ngram_range=(1,1), **parameters)
# joblib.dump(ug_vec, '/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_ug_vec.pkl')

# make bigram counter
bg_vec = CountVectorizer(ngram_range=(2,2), **parameters)
# joblib.dump(bg_vec, '/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_bg_vec.pkl')

ug_count = ug_vec.fit_transform(lst_reviews)
bg_count = bg_vec.fit_transform(lst_reviews)

print "ug =", ug_count.shape, '|', "bg =", bg_count.shape

#print "ug =", ug_count, "bg =", bg_count
#print X_trn[0,0]
# ____reviews, ____unique words

ug = (910484, 117338) | bg = (910484, 4144948)


In [24]:
## Store Trained Vectorizers ###

# store unigram counter
joblib.dump(ug_vec, '/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_ug_vec_restfood.pkl')

# store bigram counter
joblib.dump(bg_vec, '/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_bg_vec_restfood.pkl')

['/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_bg_vec_restfood.pkl']

In [41]:
## Get bigram counts ###

# Get vector of total counts, then zip feature index to bigram words
lst_tpl_bg_totals = (zip(bg_vec.get_feature_names(), np.asarray(bg_count.sum(axis=0)).ravel())) # --> 1xN list of tpl (bigram, count)

# print len(lst_tpl_bg_totals), lst_tpl_bg_totals[0:5]

# Order list of (bigram, count) tuples // sorts in place
# lst_tpl_bg_totals = sorted(lst_tpl_bg_totals, reverse=True, key=lambda x: x[1])  # interesting but unnecessary 
print len(lst_tpl_bg_totals), lst_tpl_bg_totals[0:5]


4144948 [(u'00 00', 275), (u'00 000', 4), (u'00 00am', 3), (u'00 00pm', 13), (u'00 02', 3)]


In [30]:
print len(lst_tpl_bg_totals), lst_tpl_bg_totals[0:10]

4144948 [(u'go back', 44783), (u'las vegas', 42289), (u'first time', 36230), (u'pretty good', 34510), (u'really good', 34038), (u'customer service', 32480), (u'happy hour', 31659), (u'come back', 30933), (u'next time', 30186), (u'food good', 25596)]


In [120]:
# as a heroku hack, consider modeling only the top 100 bigrams, forgetting the ratio 

In [34]:
## Get unigram counts ###

# Get vector of total counts, then zip feature index to bigram word list
lst_tpl_ug_totals = (zip(ug_vec.get_feature_names(), np.asarray(ug_count.sum(axis=0)).ravel())) # --> 1xN list of tpl (bigram, count)

# Order list of (bigram, count) tuples // sorts in place
# lst_tpl_ug_totals = sorted(lst_tpl_ug_totals, reverse=True, key=lambda x: x[1])  # interesting but unnecessary
# print len(lst_tpl_ug_totals), lst_tpl_ug_totals[0:5]

dct_ug_totals = dict(lst_tpl_ug_totals)  # order doesn't matter here, so dict is good
print len(lst_tpl_ug_totals), lst_tpl_ug_totals[0:10]

 117338 [(u'good', 563555), (u'place', 538182), (u'food', 516961), (u'great', 427186), (u'like', 425680), (u'one', 370454), (u'get', 366280), (u'time', 334358), (u'service', 315716), (u'go', 303000)]


In [47]:
## Calculate Probabilities from n-gram frequency
lst_p = []
lst_bigrams = []
for tpl in lst_tpl_bg_totals:  # get count, w1 count, w2 count
    # get bigram count
    bg_cnt = tpl[1]
    
    bg_frq = float(bg_cnt)/len(lst_tpl_bg_totals)
    
    # append bigram to list
    lst_bigrams.append(tpl[0])
    
    # slice bigram into words
    lst_bg_words = tpl[0].split()

    
    # find word count
    w1 = dct_ug_totals[lst_bg_words[0]]
    w2 = dct_ug_totals[lst_bg_words[1]]
    #print tpl[0], tpl[1], w1, w2
    
    # find word probability
#     pug1 = float(w1+1200)/len(lst_tpl_ug_totals)
#     pug2 = float(w2+1200)/len(lst_tpl_ug_totals)
    pug1 = float(w1+1800)/len(lst_tpl_ug_totals)
    pug2 = float(w2+1800)/len(lst_tpl_ug_totals)
    
    # compute probability
    lst_p.append((float(bg_frq) /(pug1 * pug2)))
    
result = pd.DataFrame()
# result['bigram'] = lst_tpl_bg_totals
result['bigram'] = lst_bigrams
result['p'] = lst_p    

result.head()

,bigram,p
0,00 00,0.001960
1,00 000,0.000138
2,00 00am,0.000179
3,00 00pm,0.000599
4,00 02,0.000216


In [48]:
## Sort results by probability ratio ###    
d = dct_ug_totals
lst = range(0, len(result) )

result.sort(columns='p', axis=0, ascending=False, inplace=True)
result['v_index'] = result.index
result['index'] = lst
result.set_index(keys='index', drop=True, append=False, inplace=True)

result.head()

,bigram,p,v_index
index,,,
0,pei wei,0.423683,2624701
1,kung pao,0.387635,1958212
2,monte carlo,0.387190,2315046
3,wi fi,0.383394,4040869
4,hong kong,0.378571,1752134


In [49]:
## Show Results ###
# result.iloc[0:110,:]
print len(result.bigram.tolist()[0:100])
print result.bigram.tolist()[0:100]

100
[u'pei wei', u'kung pao', u'monte carlo', u'wi fi', u'hong kong', u'wal mart', u'prix fixe', u'amuse bouche', u'ami gabi', u'du soleil', u'panna cotta', u'surf turf', u'tex mex', u'los angeles', u'huevos rancheros', u'hustle bustle', u'wolfgang puck', u'krispy kreme', u'foie gras', u'lotus siam', u'coca cola', u'bok choy', u'loco moco', u'tikka masala', u'flip flops', u'tater tots', u'tres leches', u'cole slaw', u'pina colada', u'banh mi', u'pinot noir', u'joel robuchon', u'butternut squash', u'gordon ramsay', u'planet hollywood', u'clam chowder', u'hush puppies', u'croque madame', u'bloody mary', u'kool aid', u'http www', u'osso bucco', u'pf changs', u'creme brulee', u'fro yo', u'beaten path', u'carne asada', u'mon ami', u'wicked spoon', u'nom nom', u'bo hue', u'jean philippe', u'cave creek', u'mani pedi', u'knick knacks', u'dim sum', u'penn teller', u'washer dryer', u'scantily clad', u'santa fe', u'bang buck', u'pun intended', u'moscow mule', u'saving grace', u'shabu shabu', u'ho

### Time to Test

In [128]:
## Load Vectorizer Models & Vectorize Test Data ###

#    load unigram counter
ug_vec = joblib.load('/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_ug_vec.pkl')
#    load bigram counter
bg_vec = joblib.load('/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q4_bg_vec.pkl')

# Count unigrams and bigrams
ug_count = ug_vec.fit_transform(X_tst)
bg_count = bg_vec.fit_transform(X_tst)

print "ug =", ug_count.shape, '|', "bg =", bg_count.shape

ug = (322, 1917) | bg = (322, 980)


In [129]:
## Arrange bigrams by count ###

# Get vector of total counts, then zip feature index to bigram words
lst_tpl_bg_totals = (zip(bg_vec.get_feature_names(), np.asarray(bg_count.sum(axis=0)).ravel())) # --> 1xN list of tpl (bigram, count)

# Order list of (bigram, count) tuples // sorts in place
lst_tpl_bg_totals = sorted(lst_tpl_bg_totals, reverse=True, key=lambda x: x[1]) 
print len(lst_tpl_bg_totals), lst_tpl_bg_totals[0:5]

980 [(u'fish fry', 24), (u'go back', 16), (u'good food', 16), (u'ice cream', 16), (u'pretty good', 15)]


In [130]:
## Get unigram counts ###

# Get vector of total counts, then zip feature index to bigram word list
lst_tpl_ug_totals = (zip(ug_vec.get_feature_names(), np.asarray(ug_count.sum(axis=0)).ravel())) # --> 1xN list of tpl (bigram, count)
print len(lst_tpl_ug_totals), lst_tpl_ug_totals[0:5]

dct_ug_totals = dict(lst_tpl_ug_totals)  # order doesn't matter here, so dict is good
print dct_ug_totals.items()[0:5]

1917 [(u'00', 3), (u'10', 24), (u'100', 3), (u'1000', 2), (u'14', 3)]
[(u'limited', 7), (u'yellow', 5), (u'four', 13), (u'prices', 26), (u'asian', 9)]


In [133]:
## Calculate Probabilities
lst_p = []
for tpl in lst_tpl_bg_totals:  # get count, w1 count, w2 count
    # get bigram count
    bg_cnt = tpl[1]
    
    # slice string key
    lst_bg_words = tpl[0].split()

    # find word count
    w1 = dct_ug_totals[lst_bg_words[0]]
    w2 = dct_ug_totals[lst_bg_words[1]]
    #print tpl[0], tpl[1], w1, w2

    # compute probability
    lst_p.append(float(bg_cnt) /(w1 * w2))
    
import pandas as pd
result = pd.DataFrame()
result['bigram'] = lst_tpl_bg_totals
result['p'] = lst_p    
    
result.sort(columns='p', axis=0, ascending=False, inplace=True)
result['v_index'] = result.index
result['index'] = range(0,len(result))
result.set_index(keys='index', drop=True, append=False, inplace=True)

result.iloc[0:100,:]

,bigram,p,v_index
index,,,
0,"(saving grace, 2)",0.500000,820
1,"(comedor guadalajara, 2)",0.500000,412
2,"(ella deli, 2)",0.500000,461
3,"(mint oreo, 2)",0.333333,678
4,"(walking distance, 2)",0.333333,942
5,"(ass governor, 2)",0.333333,343
6,"(wood paneling, 2)",0.333333,966
7,"(lo mein, 2)",0.333333,639
8,"(dane county, 3)",0.250000,183


In [ ]:
carne asada, 2)	0.500000	1550
1	(tiki torches, 2)	0.500000	10503
2	(carb loading, 2)	0.500000	1541
3	(bees knees, 2)	0.500000	989
4	(latina hussie, 2)	0.500000	5549
5	(mardi gras, 2)	0.500000	6351
6	(salsiccia rossa, 2)	0.500000	8919
7	(hush puppies, 2)	0.500000	5233
8	